In [1]:
import sys 
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 

print("Numpy version: ", np.__version__)
print("Scipy version: ", sp.__version__)

from non_Hermitian_random_matrix import non_hermitian_symmetry_list, gap_types, is_complexification
from non_Hermitian_random_matrix import DeformHamiltonian, DeformHamiltonianGenerator

from machine_learning import Cluster
from topology_table import DeformTopologicalModel
from functools import partial
import itertools

from multiprocessing import Pool
from tools import save_data, save_data_M, load_data, load_data_M

Numpy version:  1.23.5
Scipy version:  1.11.1


# Run in parallel

In [2]:
n_sample = 500
n_band = 8
n_mesh = 1 # very fast mode
is_inverse_parity = False

def sim_func_Q(Q1, Q2):
    nQ = len(Q1)
    for n in range(nQ):
        _Q = Q1[n]+Q2[n]
        for v in np.linalg.eigvalsh(_Q):
            if np.log10(np.abs(v)) < -10:
                return 0
    return 1

def get_kpoints(n_dim, n_mesh):
    '''
    Get the k points
    '''
    if n_dim == 1:
        kpoints = [[k] for k in np.linspace(0, np.pi, n_mesh)]
    else:
        values = [np.linspace(0, np.pi, n_mesh) for i in range(n_dim)]
        kpoints = list(itertools.product(*values))

    return kpoints

def fast_similarity_func(model1, model2, n_dim, n_mesh=3):
    kpoints = get_kpoints(n_dim, n_mesh=n_mesh)
    Q1 = model1.calculate_Q(kpoints)
    Q2 = model2.calculate_Q(kpoints)
    return sim_func_Q(Q1, Q2)

def worker(parameter, n_band, n_sample, n_mesh, is_inverse_parity):
    non_Hermitian_symmetry_class, n_dim, gap_type = parameter
    
    filename = "./data/data_{symmetry_class}_{gap_type}_dim_{n_dim}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)

    flag = False
    try: 
        _x, _y = load_data(filename)
    except:
        flag = True

#     print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
#         sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

    if flag:
        generator  = DeformHamiltonianGenerator(
                n=n_band, n_dim=n_dim, 
                non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
                is_inverse_parity = is_inverse_parity,
                verbose=False)
        hamiltonians = generator.generate(n_sample=n_sample)
        models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
        
        similarity_func = partial(fast_similarity_func, n_dim=n_dim, n_mesh=n_mesh)
        cluster = Cluster(similarity_function=similarity_func)
        center_indices, group_number = cluster.fit(models)
        
        #filename = "./data_P/data_{symmetry_class}_{gap_type}_dim_{n_dim}_P.h5".format(
        #    symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)
        save_data(center_indices, group_number, filename)
    
n_dims = range(1, 9+1)
parameters = list(itertools.product(non_hermitian_symmetry_list, n_dims, gap_types))
print("Total case: ", len(parameters))

_worker = partial(worker, n_band=n_band, n_sample=n_sample, n_mesh=n_mesh, is_inverse_parity=is_inverse_parity)

n_core = 32
with Pool(n_core) as pool:
    pool.map(_worker, parameters)

Total case:  1026


/tmp/ipykernel_53072/3807818536.py:11: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
